### Cella 0: stato iniziale

Questa cella stampa la versione di Python e di tutte le librerie in uso nell'ambiente Colab, quindi effettua un chek di compatibilità librerie. Questo è utile per la risoluzione dei problemi di dipendenza.

In [ ]:
# Cella 0

from pathlib import Path
import os, sys
import shutil
print("Versione del file 0.3.1, pushata in github\n Versione Python:")
print(sys.version)
print(f"PATH (Eseguibili):\n {os.environ['PATH']}")
print(f"sys.path (Librerie Python):\n {sys.path}")
print(Path.cwd())
# !pip list # decommentare per leggere l'elenco di librerie preinstallate
# in caso di anomalia del seguente check
!pip check

# ==============================================================================
# Fase 1: RILEVAZIONE AMBIENTE E SETTAGGIO BRANCH
# ==============================================================================
# 1.1 Logica di rilevazione
if Path("/content").exists():
    ENV = "COLAB"
    ROOT = Path("/content")
    BRANCH = "sviluppo"
elif Path("/home/studio-lab-user").exists():
    ENV = "SAGEMAKER"
    ROOT = Path("/home/studio-lab-user")
#   BRANCH = "main" da ripristinare quando collaudato
    BRANCH = "sviluppo"
else:
    ENV = "UNKNOWN"
    ROOT = Path(os.getcwd())
    BRANCH = "main"
    print("⚠️ Non sei nella Root di un ambiente conosciuto.")

# 1.2 Variabili derivate
# Usiamo 'repo' e 'modules' come nomi standard per facilitare la gestione delle celle future.
REPO_LOCAL = ROOT / "repo"        # Dove scaricheremo tutto il repository
TARGET_MODULES = ROOT / "modules"    # Dove copieremo i moduli per Python
REPO_URL   = "https://github.com/Theridel/Coach_2.0.git"

# 1.4 Output unico di riepilogo
print(f"{'='*40}")
print(f"🌍 AMBIENTE : {ENV}")
print(f"🌿 BRANCH   : {BRANCH}")
print(f"📁 ROOT     : {ROOT}")
print(f"📂 REPO     : {REPO_LOCAL}")
print(f"📂 MODULES  : {TARGET_MODULES}")
print(f"{'='*40}")
print("puoi scaricare il repository")

### Cella 1: Scarica il repository da github e Installazione delle dipendenze

1. **Scaricamento del repository**: tutto il repository viene copiato nella directory /content/repo. Da questo, copiamo il contenuto di /runtime su /content
2. **Installazione di varie librerie da requirements.txt**: Vengono installate librerie essenziali come `llama-cpp-python`, per l'interazione con i modelli GGUF (come Gemma). L'opzione `--upgrade` assicura di avere l'ultima versione e `-q` la rende silenziosa.

In [ ]:
# Cella 1

# ==============================================================================
# Fase 2: Scarica il repository da github
# ==============================================================================

# 2.1 Pulizia della directory (Strategica per Colab e SageMaker)
if ENV == "COLAB":
    print(f"[CLEAN] Pulizia totale Colab (eccetto Drive)...")
    for item in ROOT.iterdir():
        if item.name == "drive":
            continue
        try:
            if item.is_dir():
                shutil.rmtree(item)
            else:
                item.unlink()
        except Exception as e:
            print(f"Errore: {e}")

elif ENV == "SAGEMAKER":
    print(f"[CLEAN] SageMaker: Pulizia selettiva per evitare accumulo residui...")
    # Definiamo cosa vogliamo che sia SEMPRE pulito prima di un nuovo "travaso"
    pulizia_target = [REPO_LOCAL, TARGET_MODULES]

    for cartella in pulizia_target:
        if cartella.exists():
            print(f"   - Rimozione residui: {cartella.name}")
            shutil.rmtree(cartella)

print(f"[INFO] Pronto per Fase 2 (Clone) su {REPO_LOCAL}")

# 2.2: Clone del repository Git.
# Esegue un 'git clone' per scaricare il repository per la prima volta.
print("[PACK] Clono il repo...")
os.system(f"git clone -b {BRANCH} {REPO_URL} {REPO_LOCAL}") # Clona il repo usando os.system

# ==============================================================================
# Fase 3: Installa le dipendenze, copia del contenuto del repository
# a ROOT, /content/ per COLAB e /studio-lab-user per Sagemaker.
# ==============================================================================

# 3.1 copia da /repo
# La variabile 'src' punta alla directory 'runtime' all'interno del repository clonato.
src = REPO_LOCAL  # Puntiamo direttamente alla cartella principale del repo

# Viene eseguito un controllo per assicurarsi che la cartella 'runtime' esista.
# Ogni elemento (file o directory) all'interno di 'runtime' viene copiato direttamente in /content/.
# Se è una directory, usa shutil.copytree; altrimenti, usa shutil.copy2 per i file.

for item in src.iterdir():
    if item.name == ".git": continue # <--- Aggiungi questa riga
    target = ROOT / item.name
    if item.is_dir(): # <--- Deve essere allineato sotto 'target'
        if target.exists():
            shutil.rmtree(target)
        shutil.copytree(item, target)
    else:
        shutil.copy2(item, target) # <--- Ricordati di aggiungere questo per i file singoli!

print("[PACK] Copia completata: runtime/* → /content/")

# 3.2 Installa le librerie necessarie
%pip install -r {REPO_LOCAL}/requirements.txt --upgrade
print("Dipendenze installate da requirements.txt.")

%pip install -r {REPO_LOCAL}/requirements_AI.txt --upgrade
print("Dipendenze installate da requirements_AI.txt.")


# 3.3: Copia selettivadella directory /modules

# Definiamo sorgente e destinazione per i moduli
SORGENTE_MODULES = REPO_LOCAL / "modules"
# TARGET_MODULES è già definita come ROOT / "modules" nella tua Fase 1.2

if SORGENTE_MODULES.exists() and SORGENTE_MODULES.is_dir():
    print(f"[PACK] Sincronizzazione moduli: {SORGENTE_MODULES} → {TARGET_MODULES}")

    # Se la cartella di destinazione esiste già, la rimuoviamo per garantire una copia pulita
    if TARGET_MODULES.exists():
        shutil.rmtree(TARGET_MODULES)

    # Copia ricorsiva di tutta la struttura (file .py, __init__.py e sottocartelle)
    shutil.copytree(SORGENTE_MODULES, TARGET_MODULES)

    # Aggiunta al path di sistema per permettere l'import immediato
    if str(TARGET_MODULES) not in sys.path:
        sys.path.append(str(TARGET_MODULES))
        print(f"✅ Moduli pronti per l'import e aggiunti al sys.path")
else:
    print("⚠️ Attenzione: La directory /modules non è stata trovata nel repository clonato.")

print("Pronto per SETUP (cella 2). Puoi controllare il download con la cella Test")

# Cella 2  di comodo:
Cella 2: Setup
Questa cella si occupa della configurazione iniziale dell'ambiente.

## crea variabile dizionario "ambiente"


Da spostare

Montaggio di Google Drive: Viene montato Google Drive nel file system di Colab. Questo è fondamentale per poter accedere ai modelli GGUF salvati.
Gestione errori: avvisa in caso di fallimento del montaggio.


In [7]:
#===============================================================================
#   Fase 4. set up
#===============================================================================
import os
from pathlib import Path

# 4.1 definisco la funzione  get_env_context()
# rileviamo percorsi e variabili utili e le inseriamo in un dizionario
def get_env_context():
    """
    Rileva l'infrastruttura e mappa i percorsi core.
    Restituisce l'oggetto 'envir' (dizionario).
    """
# Identificazione Ambiente
    if Path("/content").exists():
        env_type = "COLAB"
        root_path = Path("/content")
        git_branch = "sviluppo"
    elif Path("/home/studio-lab-user").exists():
        env_type = "SAGEMAKER"
        root_path = Path("/home/studio-lab-user")
        git_branch = "sviluppo"
    else:
        env_type = "LOCAL"
        root_path = Path(os.getcwd())
        git_branch = "main"

 # Costruzione dell'oggetto envir (La mappa delle risorse)
    contesto = {
        "ENV": env_type,
        "ROOT": root_path,
        "BRANCH": git_branch,
        "REPO_LOCAL": root_path / "repo",
        "TARGET_MODULES": root_path / "modules",

        # DATABASE 1: SQLite (Locale)
        "PATH_DB_LOCAL": root_path / "agent_instructions.db",

        # DATABASE 2: Supabase (Remoto - Segnaposto per le chiavi)
        "SB_URL": None,
        "SB_KEY": None
    }

    return contesto

# --- INIZIALIZZAZIONE GLOBALE ---
# Queando questa riga viene eseguita, rende 'envir' disponibile in tutto il notebook
envir = get_env_context()


# Cella 3 di comodo
## Uso le funzioni, nell'altro script importo i moduli

In [ ]:
# from modules.envir_manager import get_env_context

# 1. Recupero dei dati dal modulo
# envir = get_env_context()

# 2. Stampa di prova nel main per verifica visiva (come richiesto)
print(f"{'='*40}")
print(f"VERIFICA AMBIENTE:")
for chiave, valore in envir.items():
    print(f"{chiave:15} : {valore}")
print(f"{'='*40}")

# 3. Da qui in poi usi solo il dizionario ambiente
# Esempio: os.chdir(ambiente['ROOT']

### Cella 2: Setup
Questa cella si occupa della configurazione iniziale dell'ambiente.

1. **Montaggio di Google Drive**: Viene montato Google Drive nel file system di Colab. Questo è fondamentale per poter accedere ai modelli GGUF salvati.
2. **Gestione errori**: avvisa in caso di fallimento del montaggio.

In [ ]:
# Cella 2: Setup (Versione con Secrets)

# 1. Importa librerie e moduli
import os
import shutil
from google.colab import drive
from google.colab import userdata  # Necessario per i Secrets
from datetime import datetime
from supabase import create_client, Client

# 2. Monta Google Drive
print("--- 1. Monta Google Drive ---")
if not os.path.exists('/content/drive'):
    try:
        drive.mount('/content/drive')
        print("✅ Google Drive montato con successo.")
    except Exception as e:
        print(f"❌ ERRORE: Impossibile montare Google Drive. {e}")

# --- CONFIGURAZIONE SUPABASE (SICURA) ---
# Recupero dai Secrets di Colab.
# Nota: In un ambiente locale potresti usare un file .env per gestire queste variabili.
try:
    SUPABASE_URL = userdata.get('SUPABASE_URL')
    SUPABASE_ANON_KEY = userdata.get('SUPABASE_ANON_KEY')
    print("🔑 Chiavi caricate dai Secrets di Colab.")
except Exception as e:
    print(f"❌ Errore nel caricamento dei Secrets: {e}")
    print("Assicurati di aver aggiunto SUPABASE_URL e SUPABASE_ANON_KEY nel pannello Secrets.")

# --- CONFIGURAZIONE PERCORSI SQLITE ---
NOME_DB = "agent_instructions.db"
CARTELLA_DRIVE = "/content/drive/MyDrive/LLM Database/"
PATH_REPO = f"/content/repo/{NOME_DB}"
PATH_DRIVE = os.path.join(CARTELLA_DRIVE, NOME_DB)
PATH_LAVORO = f"/content/{NOME_DB}"

def setup_ambiente():
    # --- Inizializzazione Supabase (Corretta) ---
    global supabase
    try:
        supabase = create_client(SUPABASE_URL, SUPABASE_ANON_KEY)
        print("✅ Connessione a Supabase stabilita.")
    except Exception as e:
        print(f"⚠️ Errore connessione Supabase: {e}")

    # --- Gestione SQLite (Tua logica originale) ---
    os.makedirs(CARTELLA_DRIVE, exist_ok=True)
    esiste_drive = os.path.exists(PATH_DRIVE)
    esiste_repo = os.path.exists(PATH_REPO)

    if esiste_drive:
        print(f"📥 Trovato database su Drive. Caricamento in corso...")
        shutil.copy2(PATH_DRIVE, PATH_LAVORO)
    elif esiste_repo:
        print(f"📦 Drive vuoto. Caricamento dal Repo GitHub...")
        shutil.copy2(PATH_REPO, PATH_LAVORO)
    else:
        print("🆕 Nessun database SQLite trovato. Ne verrà creato uno nuovo.")

    if esiste_drive and esiste_repo:
        mtime_drive = os.path.getmtime(PATH_DRIVE)
        mtime_repo = os.path.getmtime(PATH_REPO)
        if abs(mtime_drive - mtime_repo) > 2:
            print(f"⚠️ AVVISO: Differenza tra Drive e Repo rilevata.")

    print(f"✅ Ambiente pronto. File di lavoro locale: {PATH_LAVORO}")

setup_ambiente()

## Cella test, verifichiamo:

*   lo scaricamento del repository
*   La variabile d'ambiente


In [ ]:
from pathlib import Path

# Recupero sicuro del path
content_path = envir.get('ROOT')

# Itera sugli elementi nella root
for item in sorted(content_path.iterdir()):
    # Esclusione specifica per il repo (troppo vasto da listare)
    if item.is_dir() and item.name == "repo":
        print(f"📁 {item.name}/ [Directory Repo - Contenuto Nascosto]")
        continue

    # Gestione Cartelle
    if item.is_dir():
        print(f"📁 {item.name}/")
        sub_items = sorted(list(item.iterdir()))
        if not sub_items:
            print("    └── (vuota)")
        else:
            for sub_item in sub_items:
                # Mostriamo solo i file o le sottocartelle immediate
                prefisso = "    ├──" if sub_item != sub_items[-1] else "    └──"
                tipo = "📁 " if sub_item.is_dir() else "📄 "
                print(f"{prefisso} {tipo}{sub_item.name}")

    # Gestione File nella Root
    elif item.is_file():
        print(f"📄 {item.name}")

print(f"{'-'*45}")

# puliamo la chache delle librerie installate.
%pip cache purge

Test di supabase

In [ ]:
# Test  di Supabase
# lettura Direttive AI

def carica_direttive(progetto="AI-Agent"):
    try:
        # Interroga Supabase filtrando per il tuo progetto
        response = supabase.table("direttive_ai").select("*").eq("progetto_nome", progetto).execute()

        if response.data:
            print(f"--- Direttive caricate per il progetto: {progetto} ---")
            for record in response.data:
                print(f"[{record['sezione']}]: {record['contenuto']}")
        else:
            print("⚠️ Nessuna direttiva trovata per questo progetto.")

    except Exception as e:
        print(f"❌ Errore durante la lettura: {e}")

# Esegui il test
carica_direttive()

### 3. Gestione delle Istruzioni tramite Database SQLite

Per scenari più complessi, dove le istruzioni sono molteplici, dinamiche o devono essere associate a specifici contesti/tool, un database offre maggiore flessibilità e organizzazione rispetto a semplici file di testo. Qui useremo **SQLite**, un database leggero e basato su file, ideale per iniziare in Colab.

In [ ]:
# Cella 3: SQLite Operazioni e Backup
import sqlite3
import pandas as pd
# Nota: PATH_LAVORO e PATH_DRIVE sono ereditati dalla cella precedente

print("--- 2. Esecuzione Operazioni Database ---")

def gestisci_database():
    # 1. Connessione al database LOCALE (veloce e sicuro)
    try:
        conn = sqlite3.connect(PATH_LAVORO)
        cursor = conn.cursor()
        print(f"✅ Connesso al DB locale: {NOME_DB}")
    except Exception as e:
        print(f"❌ ERRORE Connessione: {e}")
        return

    try:
        # 2. Creazione Tabella (Schema)
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS instructions (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                name TEXT NOT NULL UNIQUE,
                content TEXT NOT NULL,
                description TEXT
            )
        ''')

        # 3. Inserimento Dati (Logica upsert/ignore)
        istruzioni_da_inserire = [
            ('persona_base', 'Sei un assistente AI neutrale e collaborativo. Rispondi sempre in italiano.', 'Prompt base'),
            ('formato_codice', 'Presenta sempre il codice formattato correttamente.', 'Formattazione'),
            ('evita_saluti', 'Evita saluti, vai dritto al punto.', 'Stile')
        ]

        changes_count = 0
        for name, content, desc in istruzioni_da_inserire:
            try:
                # Usiamo INSERT OR IGNORE o gestiamo l'errore per evitare duplicati
                cursor.execute("INSERT INTO instructions (name, content, description) VALUES (?, ?, ?)",
                               (name, content, desc))
                print(f"   ➕ Istruzione inserita: '{name}'")
                changes_count += 1
            except sqlite3.IntegrityError:
                # Se esiste già, possiamo decidere se ignorare o aggiornare. Qui ignoriamo e avvisiamo.
                # Se volessi aggiornare useresti: UPDATE instructions SET content = ? WHERE name = ?
                pass
                # print(f"   (i) Istruzione '{name}' già presente.")

        conn.commit()
        print(f"✅ Modifiche salvate in locale ({changes_count} nuovi inserimenti).")

        # 4. Lettura e Controllo
        print("\n--- Anteprima Contenuto DB ---")
        df = pd.read_sql_query("SELECT name, description FROM instructions", conn)
        print(df)
        print("-" * 30)

    except Exception as e:
        print(f"❌ ERRORE durante le query: {e}")
        conn.rollback()
    finally:
        conn.close()

    # 5. SALVATAGGIO SU DRIVE (Cruciale)
    # Sovrascriviamo il file su Drive con la versione appena modificata
    try:
        shutil.copy2(PATH_LAVORO, PATH_DRIVE)
        print(f"\n💾 BACKUP RIUSCITO: Database salvato su Google Drive.")
        print(f"   Percorso: {PATH_DRIVE}")
        print(f"   Ora: {datetime.now().strftime('%H:%M:%S')}")
    except Exception as e:
        print(f"\n❌ ERRORE BACKUP SU DRIVE: {e}")
        print("⚠️ Attenzione: i dati sono salvati solo nell'ambiente temporaneo di Colab!")

# Eseguiamo la funzione
gestisci_database()


### Cella 4: Caricamento e Inizializzazione del Modello LLM

Questa cella è dedicata alla fase di caricamento e inizializzazione del modello di linguaggio (LLM) Gemma.

1.  **Istanziazione:** La libreria `llama_cpp` viene impiegata per creare un'istanza dell'oggetto `Llama`.
2.  **Localizzazione Risorsa:** Si definisce il percorso su Google Drive (`CARTELLA_MODELLI`) dove risiede il file del modello (`.gguf`).
3.  **Ottimizzazione Hardware:** Il parametro `n_gpu_layers=-1` configura il modello per delegare la massima parte dei calcoli alla GPU, incrementando le prestazioni.
4.  **Gestione Contesto:** `n_ctx` imposta la finestra di contesto, cioè la quantità di testo che il modello può "ricordare" per generare risposte coerenti.

**Verifica Esito:** Un caricamento successo è attestato da un messaggio di conferma. Eventuali errori indicano problemi di percorso o compatibilità, impedendo l'operatività del sistema AI.


In [ ]:
# Cella 4: Definizione Percorso e Caricamento del Modello
print("--- 2. Caricamento Modello ---")

# 1. Creazione istanza Llama
from llama_cpp import Llama

# 2. Definizione percorso modelli su Drive
CARTELLA_MODELLI = "/content/drive/MyDrive/LLM Models/"
NOME_FILE_GGUF = "gemma-2-2b-it-Q8_0.gguf"
PERCORSO_MODELLO_COMPLETO = os.path.join(CARTELLA_MODELLI, NOME_FILE_GGUF)

llm = None # Inizializza la variabile llm

# Verifica l'esistenza del file e carica il modello
if not os.path.exists(PERCORSO_MODELLO_COMPLETO):
    print("\n" + "="*50)
    print(f"❌ ERRORE: Il file GGUF NON è stato trovato al percorso:")
    print(f"Percorso cercato: {PERCORSO_MODELLO_COMPLETO}")
    print("Controlla che il file GGUF e il nome siano corretti.")
    print("="*50)
else:
    print(f"✅ File del modello trovato: {PERCORSO_MODELLO_COMPLETO}")
    try:
        print("\nCaricamento del modello in corso... (Questo è il passo che richiede più tempo: circa 2 minuti)")
        llm = Llama(
            model_path=PERCORSO_MODELLO_COMPLETO,
            n_gpu_layers=-1, # Offload completo sulla GPU
            n_ctx=4096,
            verbose=False
        )
        print("🎉 Modello Gemma caricato e pronto per l'inferenza!")
    except Exception as e:
        print("\n" + "="*50)
        print(f"❌ ERRORE GRAVE durante il caricamento del modello. Dettagli: {e}")
        print("Controlla la configurazione della GPU e la validità del file GGUF.")
        print("="*50)

### Cella 5

In [ ]:
# Cella 5: Definizione della funzione di Inferenza
print("--- 3. Definizione Funzione ---")

def generate_response(llm_model, user_prompt, max_tokens=2024, temperature=0.7, stop_sequence=["<end_of_turn>"]):
    """
    Funzione riutilizzabile per generare risposte da un modello Gemma Instruct.
    """
    if llm_model is None:
        return "Errore: Il modello LLM non è stato caricato correttamente nella Cella 2."

    # Formattazione specifica per Gemma Instruct
    prompt_formattato = f"<start_of_turn>user\n{user_prompt}<end_of_turn>\n<start_of_turn>model\n"

    output = llm_model(
        prompt_formattato,
        max_tokens=max_tokens,
        temperature=temperature,
        stop=stop_sequence,
        echo=False
    )

    # Estrai il testo e rimuovi spazi extra
    return output["choices"][0]["text"].strip()

print("✅ Funzione 'generate_response' definita. Pronto per la Cella 6.")

### Cella 6

In [ ]:
# Cella 6: fai la domanda e ottieni la risposta
print("--- 4. Esecuzione Test ---")

# Definisci la tua domanda qui
PROMPT_UTENTE = "Non mi hai risposto alla prima domanda: ricordi le domande che ti ho già fatto? Se si, è cambiato qualcosa nel mondo e voglio vedere se tu (un llm fermo ad una data) eri in grado di prevederlo"

print(f"Domanda inviata a Gemma: {PROMPT_UTENTE}")

# Chiama la funzione
risposta = generate_response(llm, PROMPT_UTENTE)

# Stampa il risultato
print("\n" + "="*50)
print("--- RISPOSTA GENERATA ---")
print(risposta)
print("="*50)

### 7. Creazione di un file di istruzioni (esempio)

In [ ]:
# Cella 7
# Definisci il percorso e il nome del file di istruzioni
FILE_ISTRUZIONI = "istruzioni_personalizzate.txt"

# Contenuto delle istruzioni
# Questo è ciò che il tuo modello "leggerà" prima del tuo prompt principale
contenuto_istruzioni = (
    "Sei un assistente AI amichevole e collaborativo. "
    "Rispondi sempre in italiano. \n"
    "Fornisci risposte dettagliate e utili, ma sii conciso quando possibile."
    "Presenta sempre il codice formattato correttamente con i blocchi di codice. \n"
    "Evita di aggiungere saluti o ringraziamenti."
    "Rispondi direttamente alla domanda dell'utente."
)

# Scrivi le istruzioni nel file
with open(FILE_ISTRUZIONI, "w", encoding="utf-8") as f:
    f.write(contenuto_istruzioni)

print(f"File '{FILE_ISTRUZIONI}' creato con successo.\nContenuto:\n{contenuto_istruzioni}")

In [ ]:
# cella 8:
# Leggi le istruzioni dal file
try:
    with open(FILE_ISTRUZIONI, "r", encoding="utf-8") as f:
        istruzioni_da_file = f.read()
    print(f"Istruzioni lette dal file '{FILE_ISTRUZIONI}'.")
except FileNotFoundError:
    istruzioni_da_file = ""
    print(f"❌ ERRORE: File di istruzioni '{FILE_ISTRUZIONI}' non trovato.")

# Prompt principale dell'utente
PROMPT_UTENTE_AGGIUNTIVO = "crea un programma in python per montare google drive"

# Combina le istruzioni con il prompt dell'utente
# Puoi decidere come strutturare il prompt combinato.
# Ad esempio, prima le istruzioni, poi la domanda specifica.
PROMPT_FINALE = f"{istruzioni_da_file.strip()}\n\nUtente: {PROMPT_UTENTE_AGGIUNTIVO}"

print(f"\nPrompt finale inviato a Gemma:\n---\n{PROMPT_FINALE}\n---")

# Chiama la funzione con il nuovo prompt combinato
risposta_con_istruzioni = generate_response(llm, PROMPT_FINALE)

# Stampa il risultato
print("\n" + "="*50)
print("--- RISPOSTA GENERATA CON ISTRUZIONI --- ")
print(risposta_con_istruzioni)
print("="*50)

# IMPORTANTE: Metti la chiusura della connessione alla fine del notebook
# o quando sai che non dovrai più accedere al DB in questa sessione,
# altrimenti potresti avere errori se provi ad accedervi di nuovo.
# conn.close()

In [ ]:
# cella 8:
# Leggi le istruzioni dal file
try:
    with open(FILE_ISTRUZIONI, "r", encoding="utf-8") as f:
        istruzioni_da_file = f.read()
    print(f"Istruzioni lette dal file '{FILE_ISTRUZIONI}'.")
except FileNotFoundError:
    istruzioni_da_file = ""
    print(f"❌ ERRORE: File di istruzioni '{FILE_ISTRUZIONI}' non trovato.")

# Prompt principale dell'utente
PROMPT_UTENTE_AGGIUNTIVO = "spiegami che tool posso inserire con facilità in un Agent AI."

# Combina le istruzioni con il prompt dell'utente
# Puoi decidere come strutturare il prompt combinato.
# Ad esempio, prima le istruzioni, poi la domanda specifica.
PROMPT_FINALE = f"{istruzioni_da_file.strip()}\n\nUtente: {PROMPT_UTENTE_AGGIUNTIVO}"

print(f"\nPrompt finale inviato a Gemma:\n---\n{PROMPT_FINALE}\n---")

# Chiama la funzione con il nuovo prompt combinato
risposta_con_istruzioni = generate_response(llm, PROMPT_FINALE)

# Stampa il risultato
print("\n" + "="*50)
print("--- RISPOSTA GENERATA CON ISTRUZIONI --- ")
print(risposta_con_istruzioni)
print("="*50)

# IMPORTANTE: Metti la chiusura della connessione alla fine del notebook
# o quando sai che non dovrai più accedere al DB in questa sessione,
# altrimenti potresti avere errori se provi ad accedervi di nuovo.
# conn.close()

### 8. Lettura del file e integrazione nel prompt

In [ ]:
# cella 8:
# Leggi le istruzioni dal file
try:
    with open(FILE_ISTRUZIONI, "r", encoding="utf-8") as f:
        istruzioni_da_file = f.read()
    print(f"Istruzioni lette dal file '{FILE_ISTRUZIONI}'.")
except FileNotFoundError:
    istruzioni_da_file = ""
    print(f"❌ ERRORE: File di istruzioni '{FILE_ISTRUZIONI}' non trovato.")

# Prompt principale dell'utente
PROMPT_UTENTE_AGGIUNTIVO = "spiegami che tool posso inserire con facilità in un Agent AI."

# Combina le istruzioni con il prompt dell'utente
# Puoi decidere come strutturare il prompt combinato.
# Ad esempio, prima le istruzioni, poi la domanda specifica.
PROMPT_FINALE = f"{istruzioni_da_file.strip()}\n\nUtente: {PROMPT_UTENTE_AGGIUNTIVO}"

print(f"\nPrompt finale inviato a Gemma:\n---\n{PROMPT_FINALE}\n---")

# Chiama la funzione con il nuovo prompt combinato
risposta_con_istruzioni = generate_response(llm, PROMPT_FINALE)

# Stampa il risultato
print("\n" + "="*50)
print("--- RISPOSTA GENERATA CON ISTRUZIONI --- ")
print(risposta_con_istruzioni)
print("="*50)

# IMPORTANTE: Metti la chiusura della connessione alla fine del notebook
# o quando sai che non dovrai più accedere al DB in questa sessione,
# altrimenti potresti avere errori se provi ad accedervi di nuovo.
# conn.close()

### 9. Integrazione delle Istruzioni dal Database e Nuovo Ciclo di Chat
Questa sezione mostra come recuperare le istruzioni dal database SQLite e integrarle in un prompt per una conversazione interattiva con l'agente AI. Questo è un passo avanti rispetto alla gestione statica delle istruzioni da file.


In [ ]:
# Cella 9: Integrazione Istruzioni dal DB in un loop di chat

# Riutilizziamo la funzione definita in Cella 6 per recuperare tutte le istruzioni
# Assicurati che 'conn' e 'cursor' siano ancora aperti dalla Cella 6. Se hai chiuso la connessione, riaprila qui.

import sqlite3 # Ensure sqlite3 is imported in this cell for robustness

# Fix: sqlite3.Connection objects do not have a 'closed' attribute.
# To handle the case where 'conn' might be closed from a previous cell
# (as `conn.close()` was called in cell 1fc547fa), we should always re-establish the connection.
# We'll explicitly close any existing connection before opening a new one,
# which is good practice to ensure a clean state, even if already closed or unusable.
if 'conn' in globals() and isinstance(globals().get('conn'), sqlite3.Connection):
    try:
        globals()['conn'].close()
        print("Existing database connection explicitly closed before re-opening.")
    except Exception as e:
        # This catch is mostly for cases where 'conn' exists but is somehow corrupted
        print(f"Warning: Could not close existing connection cleanly: {e}")
    conn = None # Reset conn to ensure a new connection is made

print("Riapro la connessione al database...")
try:
    conn = sqlite3.connect(PERCORSO_DB_COMPLETO)
    cursor = conn.cursor()
    print("✅ Connessione al database SQLite riaperta.")
except Exception as e:
    print(f"❌ ERRORE: Impossibile riaprire la connessione al database. {e}")

def get_all_instructions_content(cursor_obj):
    cursor_obj.execute("SELECT content FROM instructions ORDER BY id")
    return "\n".join([row[0] for row in cursor_obj.fetchall()])

# Recupera tutte le istruzioni dal database
db_instructions = get_all_instructions_content(cursor)

print("--- Istruzioni dal Database ---")
print(db_instructions)
print("-------------------------------")

# Iniziamo una storia di conversazione
conversation_history = []

# Funzione per generare una risposta combinando istruzioni e storia
def generate_chat_response(llm_model, user_input, history, system_instructions, max_tokens=2024, temperature=0.7, stop_sequence=["<end_of_turn>"]):
    # Costruisci il prompt combinando istruzioni, storia e input utente
    full_prompt = f"{system_instructions.strip()}\n\n"

    for role, message in history:
        if role == "user":
            full_prompt += f"<start_of_turn>user\n{message}<end_of_turn>\n"
        elif role == "model":
            full_prompt += f"<start_of_turn>model\n{message}<end_of_turn>\n"

    full_prompt += f"<start_of_turn>user\n{user_input}<end_of_turn>\n<start_of_turn>model\n"

    print(f"\nDEBUG: Prompt completo inviato al modello:\n---\n{full_prompt}\n---\n")

    output = llm_model(
        full_prompt,
        max_tokens=max_tokens,
        temperature=temperature,
        stop=stop_sequence,
        echo=False
    )
    return output["choices"][0]["text"].strip()


print("\n--- Inizia la conversazione (digita 'esci' per terminare) ---")

while True:
    user_input = input("Utente: ")
    if user_input.lower() == 'esci':
        break

    response = generate_chat_response(llm, user_input, conversation_history, db_instructions)

    print(f"Agente: {response}")

    # Aggiungi la conversazione alla storia
    conversation_history.append(("user", user_input))
    conversation_history.append(("model", response))

print("\n--- Conversazione terminata ---")

# È buona pratica chiudere la connessione al DB quando non serve più
# Fix: sqlite3.Connection objects do not have a 'closed' attribute.
# A simpler check for sqlite3 is to see if 'conn' exists and is a connection object.
if 'conn' in globals() and isinstance(globals().get('conn'), sqlite3.Connection):
    try:
        globals()['conn'].close()
        print("Connessione al database chiusa.")
    except Exception as e:
        print(f"Warning: Could not close connection during final cleanup: {e}")
